<p style="text-align:center">
    <a href="https://skills.network" target="_blank">
    <img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/assets/logos/SN_web_lightmode.png" width="200" alt="Skills Network Logo">
    </a>
</p>


# **Black Jack  Reinforcement Learning and Monte Carlo Methods**


Estimated time needed: **60** minutes


In this project, we will try to find the best playing strategy for Black Jack by using reinforcement learning. You will explore the basics of Reinforcement Learning and Monte Carlo Methods. You will learn about training your own agent to succeed in simple and complex games.  Discover better ways to train your agent and analyze its performance.


## Objectives

After completing this lab you will be able to:

*   Work with an OpenAI Gym environments
*   Explain what Reinforcement Learning is
*   Explain what Monte Carlo Method is
*   Create an agent that uses Monte Carlo Method to play Black Jack
*   Train and Test the agents using the Black Jack environment


----


## __Table of Contents__

<ol>
    <li><a href="#Objectives">Objectives</a></li>
    <li>
        <a href="#Setup">Setup</a>
        <ol>
            <li><a href="#Installing-Required-Libraries">Installing Required Libraries</a></li>
            <li><a href="#Importing-Required-Libraries">Importing Required Libraries</a></li>
            <li><a href="#Defining-Helper-Functions">Defining Helper Functions</a></li>
        </ol>
    </li>
    <li>
        <a href="#What's-Reinforcement-Learning?">What's Reinforcement Learning?</a>
        <ol>
            <li><a href="#Basic-Terminology">Basic Terminology</a></li>
            <li><a href="#Reinforcement-Learning-Process">Reinforcement Learning Process</a></li>
        </ol>
    </li>
    <li><a href="#BlackJack-Environment">Blackjack Environment</a></li>
    <li><a href="#Epsilon-Greedy-Policy">Epsilon-Greedy Policy</a></li>
    <li><a href="#Monte-Carlo-Method">Monte Carlo Method</a></li>
    <li><a href="#Exercises">Exercises</a>
</ol>


## Setup


For this lab, we will be using the following libraries:

*   [`pandas`](https://pandas.pydata.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for managing the data.
*   [`numpy`](https://numpy.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for mathematical operations.
*   [`sklearn`](https://scikit-learn.org/stable/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for machine learning and machine-learning-pipeline related functions.
*   [`seaborn`](https://seaborn.pydata.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for visualizing the data.
*   [`matplotlib`](https://matplotlib.org/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMML0187ENSkillsNetwork31430127-2021-01-01) for additional plotting tools.


### Installing Required Libraries

The following required libraries are pre-installed in the Skills Network Labs environment. However, if you run this notebook commands in a different Jupyter environment (e.g. Watson Studio or Ananconda), you will need to install these libraries by removing the `#` sign before `!mamba` in the code cell below.


In [ ]:
# All Libraries required for this lab are listed below. The libraries pre-installed on Skills Network Labs are commented.
# !mamba install -qy pandas==1.3.4 numpy==1.21.4 seaborn==0.9.0 matplotlib==3.5.0 scikit-learn==0.20.1 math copy
# Note: If your environment doesn't support "!mamba install", use "!pip install"

The following required libraries are __not__ pre-installed in the Skills Network Labs environment. __You will need to run the following cell__ to install them:


In [ ]:
pip install gym==0.22.0 pygame

Now let's download the file with functions that will be useful.


In [ ]:
!wget https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IND-GPXX0EP1EN/blackjackutility.py

Please, it's necessary to **RESTART THE KERNEL**. 


### Importing Required Libraries

_We recommend you import all required libraries in one place (here):_


In [ ]:
import gym
import matplotlib.pyplot as plt
import math 
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import colors
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D
import copy
import pygame
from collections import defaultdict


from blackjackutility import get_total, game_result

### Defining Helper Functions

We will define some functions to help us train the algorithm and visualize the game results. 
This function is used to plot the value function:


In [ ]:
def plot_value_function(V):
    """
    plot the estimated value function for blackjack 
    Returns:  void plots value function 
    Args:
    V: a dictionary of estimated values for blackjack 
    """
    #range of player score  
    player=[state[0]  for state in  V.keys()]
    max_player=max(player)
    min_player=min(player)
    player_range=np.arange(min_player, 22, 1)
    #range of dealer score      
    dealer=[state[1]  for state in  V.keys()]
    max_dealer=max(dealer)
    min_dealer=min(dealer)
    dealer_range=np.arange(min_dealer, 11, 1)
    #empty array for the value function, first access in the players score second  is the dealer, third is if  there  is an ace    
    V_plot=np.zeros((21-min_player+1,max_dealer-min_dealer+1,2))
    #create a mesh grid for plotting 
    X,Y = np.meshgrid(dealer_range,player_range)

    #populate an array  of values for different  scores not including losing scores 
    for (player,dealer,ace),v in V.items():
        if player<=21 and dealer<=21:
            V_plot[player-min_player,dealer-min_dealer,(1*ace)]=V[(player,dealer,ace)]

    #plot surface        
    fig, ax = plt.subplots(nrows=1, ncols=2, subplot_kw={'projection': '3d'})
    ax[0].plot_wireframe(X,Y, V_plot[:,:,0])
    ax[0].set_title('no ace')
    ax[0].set_xlabel('dealer')
    ax[0].set_ylabel('player ')
    ax[0].set_zlabel('value function')
    ax[1].plot_wireframe(X,Y, V_plot[:,:,1])
    ax[1].set_title('no ace')
    ax[1].set_xlabel('dealer')
    ax[1].set_ylabel('player ')
    ax[1].set_zlabel('value function')
    ax[1].set_title(' ace')
    fig.tight_layout()
    plt.show()

    #plot top view of the surface     
    fig, ax = plt.subplots(nrows=1, ncols=2)   
    ax[0].imshow((V_plot[:,:,0]),extent =[1,10,21,4])
    ax[0].set_title('no ace')
    ax[0].set_xlabel('dealer')
    ax[0].set_ylabel('player ')   
    im=ax[1].imshow(V_plot[:,:,1],extent =[1,10,21,4])
    ax[1].set_title('ace')
    ax[1].set_xlabel('dealer')
    fig.colorbar(im, ax=ax[1])

This function will plot blackjack policy:


In [ ]:
def plot_policy_blackjack(policy):
    """
    plot the policy for blackjack 
    Returns:  void plots policy function 
    Args:
    policy: a dictionary of estimated values for blackjack 
    """    
    #range of player score 
    player=[state[0]  for state in  policy.keys()]
    max_player=max(player)
    min_player=min(player)
    #this vale is use in RL book 
    #min_player=12
    player_range=np.arange(min_player, 22, 1)
    #range of dealer score      
    dealer=[state[1]  for state in policy.keys()]
    max_dealer=max(dealer)
    min_dealer=min(dealer)
    dealer_range=np.arange(min_dealer, 11, 1)
    #empty array for the value function, first access in the players score second  is the dealer, third is if  there  is an ace    
    policy_plot=np.zeros((21-min_player+1,max_dealer-min_dealer+1,2))
    #create a mesh grid for plotting 
    X,Y = np.meshgrid(dealer_range,player_range)
    
    
    #populate an array  of values for different  policy not including losing states above 21 
    for (player,dealer,ace),v in policy.items():
        if player<=21 and dealer<=10 and player>=min_player:
            policy_plot[player-min_player,dealer-min_dealer,(1*ace)]=policy[(player,dealer,ace)]

    
    fig, ax = plt.subplots(nrows=1, ncols=2)   
    ax[0].imshow((policy_plot[:,:,0]),cmap=plt.get_cmap('GnBu', 2),extent =[1,10,21,4])
    ax[0].set_title('no ace')
    ax[0].set_xlabel('dealer')
    ax[0].set_ylabel('player ') 
    

    ax[1].set_title('ace')
    ax[1].set_xlabel('dealer')
    im=ax[1].imshow(policy_plot[:,:,1],extent =[1,10,21,4],cmap=plt.get_cmap('GnBu', 2))
    fig.colorbar(im, ax=ax[1],ticks=[0 , 1])

This function calculates the average number of wins for a game of blackjack given a policy:


In [ ]:
def average_wins(environment,policy=None,episodes=10):
    """
    This function calculates the average number of wins for a game of blackjack given a policy.
    If no policy is provided a random policy is selected.
    Returns: average_wins: the average number of wins 
    std_wins: the average number of wins 
    Args:
    environment:AI gym balckjack envorment object 
    policy:policy for blackjack if none a random  action will be selected 
    episodes: number of episodes 
    """

    win_loss=np.zeros(episodes)

    for episode in range(episodes):
        state=environment.reset()
        done=False

        while (not(done)):
            if policy and isinstance(policy[state],np.int64):
                 
                action=policy[state]
                
            else:
                action=environment.action_space.sample()

            state,reward,done,info=environment.step(action)
        result=game_result(environment,state,show=False)
        if reward==1:
            win_loss[episode]=1
        else:
            win_loss[episode]=0  

        
    average_wins=win_loss.mean()
    std_win=win_loss.std()/np.sqrt(episodes)

    return average_wins ,std_win

## Reinforcement Learning Explained 
In general **Reinforcement Learning**  is just another machine learning method, which is based on rewarding desired actions/output and punishing for the undesired ones. Reinforcement learning models, just like people, are choosing which action to make based on the expected return of each action. You must give your model some input which include the information about current situation and possible actions, then you must reward it based on the output. Reinforcement learning models learn to perform a task through repeated trial and error interactions with a changing environment, without any human intervention.
 
### Basic Terminology 

* **Agent**: is your reinforcement learning model, it's a decision maker and learner, 

* **Environment**: is a world around your agent, the agent learns and acts inside of it. The environment takes the action provided by the agent and returns the next
**state** and the **reward**.

* **State**: is a complete description of the state of the environment. 

* **Action**: is the way agent interacts with the environment, the moves that your agent can make. **Aciton Space** is the set of all possible actions. 

* **Reward**: is the feedback from the environment, it can be negative or positive and impacts the agent and serves as an indication to an agent of what you want it to achieve. Rewards are generally unknown and agents learn how to correctly estimate the reward.

* **Policy**: is a rule used by an agent to decide what actions to take, given the specific state. It works as a map from state to some action or a set of probabilities for each action in the action space. 


* **Value Function**: is the function that returns the expected total reward your agent can get from following the specific policy. The agent uses this value function to make decisions and learns by updating the expected reward values of the parameters of this function. In this lab we will be using the  state-action value function, so our function $Q(s,a)$ will use a state-action pair and will return an estimated reward for taking action $a$ from state $s$. 

### Reinforcement Learning Process
1. Agent plays a number of games
2. In every game, the agent chooses an **Action** form the action space by using **Policy** and **Value Function**
3. **Action** impacts the environment and the **Reward** and the new **State** is returned to the agent.
4. The agent keeps track of what reward it received after choosing a certain action from a certain set. 
5. The after completing the game, the agent updates the estimated reward for each state and action by using the actual rewards values received while playing the game. 
6. The whole process repeats again.


<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IND-GPXX0O9IEN/reinforcement-learning-fig1-700.jpg" width="30%" alt="cheques image">


Famous RL models that play Chess, Go or Atari Games on superhuman levels, are all based on the aforementioned principles and concepts. 

At first, these definitions may look confusing but don't be scared, we will review and practice them more through out the lab 

<p style='color: blue'>Now that you know the basics, lets find what the aforementioned things are in the Blackjack environment.</p>


## BlackJack Environment 

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IND-GPXX0EP1EN/blackjack.gif" width="30%" alt="iris image">

**Blackjack** is a card game played against a dealer. At the start of a round, both player and dealer are dealt 2 cards. The player can only see one of the dealer’s cards. The goal of the game is to get the value of your cards as close to 21 as possible, without crossing 21. The value of each card is listed below.

* 10/Jack/Queen/King → 10
* 2 through 9 → Same value as card
* Ace → 1 or 11 (Player’s choice). Note that ace is called useful when It can be counted as 11 without going bust.

If the player has less than 21, they can choose to “hit” and receive a random card from the deck. They can also choose to “stand” and keep the cards they have. If the player exceeds 21, they go “bust” and automatically lose the round. If the player has exactly 21, they automatically win. Otherwise, the player wins if they are closer to 21 than the dealer.


Let's initialize our environment and explore it a bit.


A few notes on **Open Ai Gym**: Open Ai Gym is a toolkit for developing and comparing reinforcement learning algorithms. This is the gym open-source library, which gives you access to a standardized set of environments. It's quite useful in our case, since we are with a pretty standard environment. It also allows you to create your own custom environment but for now we will only explore a pre-define Blackjack environment. 


We create an openAI gym blackjack enviroment by calling gym method, we will use the `make` function to do so.


In [ ]:
environment= gym.make('Blackjack-v1')

Now let's see what the **observation space** for our environment is. Observation space is a set of all possible states. We can view the space using the method `observation_space` :


In [ ]:
environment.observation_space

The observation of is a 3-tuple of: the player's (you) current sum, the dealer's one showing card (1-10 where 1 is ace), and whether or not the player holds a usable ace (0 or 1) or (`True` or `False`).

    Tuple(the players current sum, dealer's one showing card player holds, usable ace (0 or 1) )
Then we can explain `Tuple(Discrete(32), Discrete(11), Discrete(2))` as
* The highest score you can achieve is (11,10,11), and the lowest is (1), so there are 32 states for the player's score.
* The dealer only shows one card, which can be anything from 1 to 11
* The 'usable ace' space is True/False, so the size of the space is 2.

So there are $32 \times 11 \times 2 = 704$ possible states. Note that we have to reset the environment to start working with it. This function resets the game environment to what it was when you started the game. It will return the initial state $S_0$:


In [ ]:
state=environment.reset()
print("s_{}={}".format(0, environment.observation_space.sample())) 

Let's see more possible states:


In [ ]:
for i in range(15):
    print("s_{}={}".format(i, environment.observation_space.sample()))

Let's check the action space of this environment, think what it should be before running the code:


In [ ]:
environment.action_space.n

Hope you guessed correctly, the answer is $2$, we can either $hit$ or $stay$: $$ A_t =0   (\textit{stay})\quad A_2 =1   \textit{(hit)}$$


In [ ]:
for t in range(10):
    action=environment.action_space.sample()
    if action:
        print("Hit, A_{}={}".format(t,action))
    else:
        print("Stay",action)

Let's also check the player and dealers cards by using `environment.player` and `environment.dealer` function 


In [ ]:
print(environment.observation_space.sample())
print(environment.player)
print(environment.dealer)

We will introduce a bit more terminology, **Episode** is a set of  agent-environment interactions from initial to final state, i.e it's one game that agent the plays. In addition, our agents are operating in a discrete-time game. Each time-advancing decision is a **step** (e.x. taking some action from some state). It's easy to see that each Episode consists of a series of steps. 

Let's play through some episodes of the game, choosing random actions for each step: 


In [ ]:
episodes=2
sum_=0
result=0
error=0

#setting the seed for reproduceability

for episode in range(episodes):
    state=environment.reset()
    done=False
    print("_________________________________________")
    print("episode {}".format(episode))
   

    print("state:",state)
    print("player has", environment.player)
    print("the players current sum: {},\n  dealer's one showing card: {}, usable ace: {}".format(state[0],state[1],state[2]))
    print('dealer', environment.dealer)
    while (not(done)):
    
        action = environment.action_space.sample()
       
        if action:
            print("hit")
        else:
            print("stay")
        
        print("action:",action)

        state,reward,done,info = environment.step(action)
    print("Done :", done)
    result=game_result(environment,state)
    sum_+=reward
sum_

Run the code above a few more times and observe that our games are not really lucky. Try playing yourself, may be you can do better than a random action generator:


In [ ]:
episodes = 5
your_return = 0
for episode in range(episodes):
    state = environment.reset()
    done = False
    print("_________________________________________")
    print("episode {}".format(episode))
    print("You are. the agent")

    print("player has", environment.player)
    print("the players current sum: {},\n  dealer's one showing card: {}, usable ace: {}".format(state[0],state[1],state[2]))
    while (not(done)):
    
        action=int(input("Press one to Hit and Zero to stay"))
        if action:
            print("hit")

        else:
            print("stay")
        
        print("action:",action)

        state,reward,done,info=environment.step(action)
        print(state)
    result=game_result(environment,state)
    your_return += reward
print("your return",your_return )

Well, I assume, as long as you are not a professional gambler, your results were not much better than random model results. What will help us to win at this Casino Game, is ironically a casino name method: **Monte Carlo Method**. But before jumping there, lets define some things that will build an infrastructure for our learning process and elaborate/ implement previously defined terminology. 


At first we will start by exploring what greedy policy is and does:


### Epsilon-Greedy Policy

If you remember, as was mentioned before, policy is just a function that defines which action our agent should take based on the current state. In our environment, a simple deterministic policy $\pi$ for the the state $(15,10,0)$ may look like: 

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IND-GPXX0EP1EN/Screenshot%202022-11-23%20at%2010.36.45%20AM.png" width="50%" alt="iris image">

Now let's clarify a few things with the title. **Epsilon**, is just some constant, ($0 \leq \epsilon \leq 1$), and it will define some probability. **Greedy**, is a concept in computer science where a greedy algorithm is the one that makes the locally optimal choice at each stage. In our case, greedy policy implies that it will choose an action with the biggest estimated return. 

For now assume that $Q(s,a)$ is our value function, it will return an **estimated** reward based on the given state and action and let $A$ be the action space. Then our policy can be simply defined: 


$$
\pi(s) =
\begin{cases}
a = max_{a^* \in A}Q(s,a^*)) & \text{with probability 1-}  \epsilon \\\\\\
\text{some }a \in A & \text{with probability } \epsilon \\\\
\end{cases}
$$


You may ask why wouldn't we always use the best action, the action with the best estimated reward, what's the point of this epsilon constant. For it we will have to learn about 2 more concepts: 
 
* **Exploration** happens when the agent takes the random action to explore more opportunities, gather more information about possible actions and the environment.
* **Exploitation** happens when the agent makes the best decision given current information, it uses the best estimated action to maximize the reward. 

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IND-GPXX0EP1EN/Screenshot%202022-11-23%20at%2010.51.11%20AM.png" width="50%" alt="iris image">

As demonstrated in the diagram above, **Epsilon** defines the trade-off between Exploration and Exploitation.  We need it because the best long-term strategy may involve short-term sacrifices and in most cases, agents must explore the environment and gather enough information to make the best overall decisions. It may save our agent from doing decisions that work instead of finding the best actions. 

Let's define a new python function that will follow the epsilon probability and return an action:


In [ ]:
def random_action(action,epsilon=0.1,n_actions=2):
    ''' 
    This function takes the best estimated action, eplsilon, and action space 
    and returns some action. 
    '''
    # generate a random number from 0 to 1.
    number = np.random.rand(1)
    
    # if number is smaller than 1-epsilon then return best estimated action
    if number<1-epsilon:
        return action
    # if number is bigger or equals to 1-epsilon then return some random action from the action space
    else:
        action=np.random.randint(n_actions)  
        return action 

Now, since we already know what **Greedy Policy**, **Episode** and **Reward** are, we are ready to learn about Monte Carlo Method + some tricks to make it better. 
Math below is may be a bit complicated, it's ok if you don't fully understand it, try understanding the general strategy of this learning algorithm and the purpose of various model parameters.


## Monte Carlo Method 
#### Let's talk about the heart of our algorithm, the Value function that we will be using and how it estimates the reward for each action given the state. 

Monte Carlo Method was invented by invented by Stanislaw Ulman in the 1940s, when trying to calculate the probability of a successful Canfield solitaire (He was sick and had nothing better to do). Ulman randomly lay the cards out and simply calculated the number of successful plays. We will apply the same approach to create our value function. The basic principle of Monte Carlo method can be summarized in 4 steps: 

1. Define the Domain of Possible inputs 
2. Generate inputs randomly from a probability distribution over the domain
3. Perform a deterministic computation on the inputs
4. Average the results

Before we can see it in action let's define a few things. Review that **Episode** is an agent-environment interactions from initial to final states which constists of steps in a in a discrete-time game. 

Monte Carlo reinforcement learning learns from **episodes of experience**, it functions by setting the value function equal to the empirical mean return.
Let's assume that we have some initialized policy $\pi$ that our agent follows. Then let's play a game once and gain the following episode: 

$$(S_1, A_1, R_1)\rightarrow(S_2, A_2, R_2)\rightarrow  (S_3, A_3, R_3)\rightarrow ...\rightarrow  (S_n) \sim \pi$$

Now let's look at an total expected reward of taking an Action $A_t$ in the state $S_t$, where t is some **time step**. 

* At **time step** $t=0$ (the first time step), the environment (including the agent) is in some state $S_t = S_0$ (the initial state), takes an action $A_t = A_0$ (the first action in the game) and receives a reward $R_t = R_0$ and the environment (including the agent) moves to a next state $S_{0+1} = S_{1}$

Let's define a function $G$, which will just give us the expected total discounted reward at each time step: 
$$G(t) = R_t +\gamma R_{t+1}+\gamma^2 R_{t+2} + ...+ \gamma^{k}R_{t+k}$$

Discount factor $\gamma \in \left[0,1\right]$ is an important constant. We add the initial return $R_1$ as it is, without modifying the value, then to get the total reward we are adding $R_{t+1}$ but note that the value is multiplied by $0\leq \gamma \leq 1$, so $R_{t+1}$ is only partially added to $R_1$, $R_{t+2}$ is multiplied by $\gamma^2$, $R_{t+3}$ is multiplied by $\gamma^3$ and so on. Gamma determines how much the reinforcement learning agents cares about rewards in the distant future relative to those in the immediate future. Note that if $\gamma=0$ then total expected return will be defined just by initial reward, so agent will only learn and care about actions that produce an immediate reward. 

Now we can define our action-value function $Q_{\pi}(S, A)$ for some sate $S$ and action $A$ as: 
$$Q_{\pi}(S, A) = E_{\pi}\left[G(t)|S_t = S, A_t = A \right ]$$

So value function returns the expected value of a total discounted reward $G(t)$ for the time step $t$ at which $S_t = S$ and $A_t = A$. So, as an example, our state-value for the start state $S = 0$ should look something like:


Now, after completing a series of episodes in the game, how can we adjust the expected values, or a bigger question is how's the learning process happening in Monte Carlo Method. For it we will use the concept of **Incremental means**. 

**Incremental means**, is just an average of values that's computed incrementally. Let $x_1, x_2,..., x_n$ be the set of values, Let $\mu_1, \mu_2, ... , \mu_{n-1}$ be an sequence of means, where $\mu_1 = x_1$, $\mu_2 = \dfrac{x_1+x_2}{2}$ and $\mu_3 = \dfrac{x_1+x_2+x_3}{3}$ and so on. Let's see how the mean is defined incrementally:

\begin{align*} 
\mu_n &= \frac{1}{n}\sum_{i = 1}^{n}x_i\\\\  
&= \frac{1}{n} (x_n +  \sum_{i = 1}^{n-1}x_i)\\
&= \frac{1}{n}(x_n +  (n-1)\mu_{n-1})\\
&= \mu_{n-1} + \frac{1}{n}(x_n - \mu_{n-1})
\end{align*}


Now we can put everything together to describe the Monte Calro Method Learning Process. Let's have an episode: 
$$(S_1, A_1, R_1)\rightarrow(S_2, A_2, R_2)\rightarrow  (S_3, A_3, R_3)\rightarrow ...\rightarrow  (S_n) \sim \pi$$

For each (state, action) pair we will keep track of the number of times this (state, action) was visited, let's define function $N(s_t,a_t)$, then every time we visit (state, action) we will update the visits counter and then adjust the running mean:
\begin{align*}
N(S_t, A_t)&+=1\\\\
Q(S_t, A_t)&+= \frac{1}{N(S_t, A_t)}(G(t) - Q(S_t, A_t))
\end{align*}

Let's look for at an example episode where we update our Q function:


<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IND-GPXX0EP1EN/Screenshot%202022-11-23%20at%2011.58.48%20AM.png" width="50%" alt="iris image">


Now we can see how in this episode, before the game begins the number of visits of this state with this action is 0, then the game starts and the number of visits functions is updated. Return is $0$ in the beginning. The action  is made, by using the predefined policy, we are changing the state and the reward is received since we have more than a dealer. The total reward is updated and the Q function is updated by calculating the average reward of making action $0$ in this state. Since it was the first time this state action pair was visited, we are just performing $1/1$ so we get $1$.


 So now we know how to update the action-value function and how to use it in combination with our policy to maximize the rewards, it can be summarized as: 
<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IND-GPXX0O9IEN/Screenshot%202022-11-08%20at%201.27.51%20PM.png" width="50%" alt="iris image">

Monte Carlo algorithm/method is a type of **model-free** reinforcement learning, since the agent does not use predictions of the environment response, so it's not trying to create a statistical **model** of the environment.

Before implementing it, we will add a few more tricks and parameters to make this  method more efficient.


## Improving Monte Carlo with First Starts:
This concept is an important strategic decision that you have to make every time you build an episode based learning algorithm. Note that every episode, we update our Q function, based on states and actions that were visited, some state-action pairs can be visited more than one per episode. **Every-Visit MC** is Monte Carlo algorithm that averages returns for every time state-action pair is visited in an episode, where **First-visit MC** averages returns only for first time state-action pair is visited in an episode. 

Let's check its implementation in the **First-visit MC** pseudo-code: 

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IND-GPXX0EP1EN/Screenshot%202022-11-23%20at%2012.33.51%20PM.png" width="50%" alt="iris image">
Where $T$ is the last step in the episode, $T-1$ is the second last one and so on.

Since, it's impossible to have the same state appearing more than once in Blackjack episode, we will only use the First-visit implementation. 


## Improving Monte Carlo with Exploring-Starts:
Note that in big and complicated environments not every (state,action) pair may be visited during the learning process. One possible solution to this problem may be adding **exploring-starts** method. In the beginning of each episode we are always starting in the initial state $S$, but with exploring-starts we will choose our starting state randomly. **Exploring-starts** is specifying that episodes start in a state–action pair, and that every pair has a nonzero probability of being selected as the start.

Let's check some pseudo code, to see how exploring-starts can be implemented in to our Monte Carlo algorithm. 

<img src="https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/IND-GPXX06PMEN/Screenshot%202022-11-09%20at%2012.24.33%20AM.png" width="50%" alt="iris image">

As you can see the main part, the only thing that changes in the code itself, is that the initial state of each episode is chosen randomly, the rest of Monte Carlo Algorithm remains unchanged.

Note that **Exploring-starts** is partially implemented in Black Jack environment by default, since player gets a random card combination in the beginning of each game, so what's left for us is to choose a random action, to generate a random initial state-action pair. 

<p style='color: blue'>Now let's implement it in our new code:</p>


In [ ]:
def monte_carlo_ES( environment,N_episodes=100000, discount_factor=1,first_visit=True, epsilon =0.1, theta=0.0001):
    """
    plot the policy for blackjack 
    Returns:  
    policy: a dictionary of estimated policy for blackjack 
    V: a dictionary of estimated values for blackjack 
    Q: a dictionary of estimated action function
    DELTA: list of deltas for each episode 
    Args:
    environment:AI gym balckjack envorment object 
    N_episodes:number of episodes 
    discount_factor:discount factor
    first_visit: select first-visit MC (Ture) and every-visit MC (False)
    epsilon: epsilon value 
    theta:stoping threshold
    """  
    #a dictionary of estimated values for blackjack 
    V=defaultdict(float)
    #a dictionary of estimated action function for blackjack
    Q=defaultdict(float)
    # number of visits to the action function 
    NumberVisitsValue= defaultdict(float)
    # visits to action function
    NumberVisits= defaultdict(float)
    #dictionary  for policy 
    policy=defaultdict(float) 
    #number  of actions 
    number_actions=environment.action_space.n
    #list of max difference between  value functions per  iteration 
    DELTA=[]

    for i in range(N_episodes):
        #max difference between  value functions
        delta=0
        #list that stores each state and reward for each episode     
        episode=[]
        # reset the  environment for the next episode and find first state  
        state=environment.reset()   
        #reward for the first state
        reward=0.0
        #flag for end of episodes  
        done=False
        #action for the first state 
        action=np.random.randint(number_actions)
        #append firt state, reward and action
        episode.append({'state':state , 'reward':reward,'action':action})
        #Past states for signal visit  Monte Carlo 
        state_action=[(state,action)]
        #enumerate for each episode 
        while not(done):

                #take action and find next state, reward and check if the episode is  done (True)
                (state, reward, done, prob) = environment.step(action)

                #check if a policy for the state exists  
                if isinstance(policy[state],np.int64):
                    #obtain action from policy
                    action=int(policy[state])
                    random_action(action,epsilon,number_actions)
                else:
                     #if no policy for the state exists  select a random  action  
                    action=np.random.randint(number_actions)
                #add state reward and action to list 
                episode.append({'state':state , 'reward':reward,'action':action})
                #add  states action this is for fist visit only 
                state_action.append((state,action))
         #reverse list as the return is calculated from the last state
        episode.reverse()
        #append the state-action pairs to a list 
        state_action.reverse()


        #determine the return
        G=0

        for t,step in enumerate(episode):

                #check flag for first visit
                G=discount_factor*G+step['reward']
                #check flag for first visit
                if first_visit:
                    #check if the state has been visited before 
                    if (step['state'],step['action']) not in set(state_action[t+1:]): 

                        #increment counter for action 
                        NumberVisits[step['state'],step['action']]+=1
                        #increment counter for value function 
                        NumberVisitsValue[step['state']]+=1
                        #if the action function value  does not exist, create an array  to store them 
                        if not(isinstance(Q[step['state']],np.ndarray) ):
                            Q[step['state']]= np.zeros((number_actions))

                        #calculate mean of action function Q Value functions V using the  recursive definition of mean 
                        Q[step['state']][step['action']]=Q[step['state']][step['action']]+(NumberVisits[step['state'],step['action']]**-1)*(G-Q[step['state']][step['action']])
                        
                        # record the old value of the value function 

                        v=V[step['state']]
                        
                        V[step['state']]=V[step['state']]+(NumberVisitsValue[step['state']]**-1)*(G-V[step['state']])
                        #update the policy to select the action fuciton argment with the largest value 
                        policy[step['state']]=np.random.choice(np.where(Q[step['state']]==Q[step['state']].max())[0])
                        #find max difference between all value functions per  iteration 
                        delta=max(delta,abs(v-V[step['state']]))


                else:
                         #increment counter for action 
                        NumberVisits[step['state'],step['action']]+=1
                        #increment counter for value function 
                        NumberVisitsValue[step['state']]+=1
                        #if the action function value  does not exist, create an array  to store them 
                        if not(isinstance(Q[step['state']],np.ndarray) ):
                            Q[step['state']]= np.zeros((number_actions))

                        #calculate mean of action function Q Value functions V using the  recursive definition of mean 
                        Q[step['state']][step['action']]=Q[step['state']][step['action']]+(NumberVisits[step['state'],step['action']]**-1)*(G-Q[step['state']][step['action']])
                        v=V[step['state']]
                        V[step['state']]=V[step['state']]+(NumberVisitsValue[step['state']]**-1)*(G-V[step['state']])
                        ##update the policy to select the action functioon argument with the largest value 
                        policy[step['state']]=np.random.choice(np.where(Q[step['state']]==Q[step['state']].max())[0])
                        #find max difference between all value functions per  iteration 
                        delta=max(delta,abs(v-V[step['state']]))
            
        DELTA.append(delta)
        if delta<theta:
            break

    return policy, V, Q,DELTA

A few more notes about the implementation, we are using a value function $V(s)$ for some state $s$ as an indication of what our model thinks the expected return is  when following the best action given by the Q function. So $V(s) = max(Q(s,a))$ for all $a$ in the action space. Our implementation returns the value function $V$ as a dictionary. $DELTA$ indicates the value by which the $V$ function for each state was updated, for example if $V_0(s) = 0.7$ and after the next episode $V_1(s) = 0.9$ then $DELTA$ is $0.9 - 0.7 = 0.2$.  The last parameter is $theta$, that indicated a stoping threshold, so if delta is very small, there is no reason to continue training the algorithm, since updates to the values are minimal we will stop the training process. 

Lets perform Monte Carlo training with 20000 episodes to see the results of our training.  


In [ ]:
 policy, V, Q,DELTA= monte_carlo_ES(environment,N_episodes=20000, discount_factor=1,epsilon = 0.1,first_visit=True,theta=0)  

Let's plot the delta value for each episode to see how the update rates are changing:


In [ ]:
plt.plot(DELTA)
plt.xlabel("episodes")
plt.ylabel("delt ")
plt.show()

As you can see the delta parameter is generally decreasing, which makes sense since our model comes closer and closer to the optimal values for the value function. Speaking of which, let's see the map of expected returns based on what player has and what the dealer has. 


In [ ]:
plot_value_function(V)

We see a general trend, as the score of the player increases the value function takes on higher values so our expected return grows. Let see the average result of playing ten thousand games. Let's compare the result, first  using the random policy.


In [ ]:
average ,std_win=average_wins(environment,episodes=10000)
print("average number of wins", average)

So the random policy gives us around $28\%$ of wins, now let's see what the trained policy returns:


In [ ]:
average ,std_win=average_wins(environment,policy,episodes=10000)
print("average wins:",average,std_win )

Oh wow, more than 10% increase, that's not bad at all, lets see what happens when we use more episodes to train our model. 


In [ ]:
policy, V, Q,DELTA = monte_carlo_ES( environment,N_episodes=50000, discount_factor=1,first_visit=True,theta=0)  

Let's see what's happening with the learning rate:


In [ ]:
plt.plot(DELTA)
plt.xlabel("episodes")
plt.ylabel("delt ")
plt.show()

Falling as expected, now let's check what our policy returns:


In [ ]:
plot_policy_blackjack(policy)

It looks like the optimal policy for blackjack is: If the agent has no ace, the higher the dealer is showing, the more likely the agent is to hit, the exception is if the dealer has an ace. If the agent has an ace, the strategy is different. The agent will stick if the sum of their cards is over 11 and, for the most part, hold the player's sum is over 18. Let's plot the value function to explore the expected returns after 50 000 episodes:


In [ ]:
plot_value_function(V)

And, most importantly, lets compare the accuracy of the results:


In [ ]:
average ,std_win=average_wins(environment,policy,episodes=10000)
print("average wins:",average,std_win )

We see as the accuracy is now approximately 43% a 15 % improvement from a random policy:


In [ ]:
accuracy = [] 
episodes = []

for n_episode in [1,50,100,500,1000,5000,10000,50000,100000]:
    print("n_episode: ", str(n_episode))
    policy, V, Q, DELTA = monte_carlo_ES( environment,N_episodes=n_episode, discount_factor=1,first_visit=True, theta = 0)  
    average ,std_win = average_wins(environment,policy,episodes=10000)
    print("n_episode: ", str(n_episode), " average: ", str(average))
    accuracy.append(average)
    episodes.append(n_episode)

We see that after 50000 episodes the improvement is negligible.


In [ ]:
plt.plot(episodes,accuracy)
plt.title("Number of wins vs number of episodes ")
plt.ylabel('percentage of wins')
plt.xlabel('number of episodes ')
plt.show()

Let's experiment with the discount factor, to see if it changes anything: 


In [ ]:
accuracy=[] 
discounts=[]

for discount in [0,0.01,0.1,0.5,1.0]:
    policy, V, Q, delta = monte_carlo_ES( environment,N_episodes=100000, discount_factor=discount,first_visit=True, theta=0)  
    average ,std_win=average_wins(environment,policy,episodes=10000)
    print("discount: ", str(discount), " average: ", str(average))
    discounts.append(discount)
    accuracy.append(average)

We see as the Discount factor increases, the percentage of wins increases as well, the rate of increase begins to slow down when the discount value is 0.6.


In [ ]:
plt.plot(discounts,accuracy)
plt.title("percentage of wins vs discount factor ")
plt.ylabel('percentage of wins')
plt.xlabel('discount factor')
plt.show()

In Conclusion, we have created a successful algorithm, that was able to give us perhaps not a winning, but an optimal strategy for blackjack. 


# Exercises

<p style='color:blue'>Here are some exercises for you to practice what you have learned.</p>

As you have noticed, there is a good amount of parameters that can be modified when training our model. Use `monte_carlo_ES` function using every-visit, use 50000 episodes and discount factor of 1


In [ ]:
#TODO: implement the code with defined parameters

<details>
    <summary>Click here for Solution</summary>

```python
policy, V, Q, delta = monte_carlo_ES( environment,N_episodes=50000, discount_factor=1,first_visit=False, theta=0)
average ,std_win=average_wins(environment,policy,episodes=10000)
```

</details>


Determine the number of wins  after 1000 episodes. 


In [ ]:
#TODO: implement the code to calculate the winnning rate

<details>
    <summary>Click here for Solution</summary>

```python
average ,std_win=average_wins(environment,policy,episodes=10000)
```

</details>


# Congratulations! - You have completed the lab


<a href="https://www.linkedin.com/in/joseph-s-50398b136/?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkCoursesIBMDeveloperSkillsNetworkML0101ENSkillsNetwork20718538-2021-01-01" target="_blank">Joseph Santarcangelo</a>

[Artem Arutyunov](https://www.linkedin.com/in/artem-arutyunov/)


## Citations:

* [OpenAi BlackJack](https://github.com/openai/gym/blob/master/gym/envs/toy_text/blackjack.py)

* [Algorithms and pseudocode: Sutton, Richard S., and Andrew G. Barto. "Reinforcement learning: An introduction](https://www.amazon.ca/Reinforcement-Learning-Introduction-Second-Paperback/dp/B0B95WFGV6/ref=asc_df_B0B95WFGV6/?tag=googleshopc0c-20&linkCode=df0&hvadid=578919340205&hvpos=&hvnetw=g&hvrand=11011609051689383259&hvpone=&hvptwo=&hvqmt=&hvdev=c&hvdvcmdl=&hvlocint=&hvlocphy=9000828&hvtargid=pla-1728961664549&psc=1)


## Change Log


|Date (YYYY-MM-DD)|Version|Changed By|Change Description|
|-|-|-|-|
|2022-11-17|0.1|Artem Arutyuno|Create Lab|


Copyright © 2022 IBM Corporation. All rights reserved.


IBM has a special offer for watsonx.ai a studio for new foundation models, generative AI and machine learning. To take advantage of this offer visit [watsonx.ai](https://dataplatform.cloud.ibm.com/registration/stepone?utm_medium=Exinfluencer&utm_source=Exinfluencer&utm_content=000026UJ&utm_term=10006555&utm_id=NA-SkillsNetwork-Channel-SkillsNetworkGuidedProjectsINDGPXX0EP1EN1318-2023-01-01&context=wx&apps=data_science_experience%2Cwatson_data_platform%2Ccos)
